In [1]:
start.time <- Sys.time()

In [ ]:
# ---------- Script ----------
# input directory
input_dir <- paste(getwd(), "/data", sep = "")
input_dir

In [2]:
library(Spectra)

Lade nötiges Paket: S4Vectors

Lade nötiges Paket: stats4

Lade nötiges Paket: BiocGenerics


Attache Paket: ‘BiocGenerics’


Die folgenden Objekte sind maskiert von ‘package:stats’:

    IQR, mad, sd, var, xtabs


Die folgenden Objekte sind maskiert von ‘package:base’:

    Filter, Find, Map, Position, Reduce, anyDuplicated, append,
    as.data.frame, basename, cbind, colnames, dirname, do.call,
    duplicated, eval, evalq, get, grep, grepl, intersect, is.unsorted,
    lapply, mapply, match, mget, order, paste, pmax, pmax.int, pmin,
    pmin.int, rank, rbind, rownames, sapply, setdiff, sort, table,
    tapply, union, unique, unsplit, which.max, which.min



Attache Paket: ‘S4Vectors’


Die folgenden Objekte sind maskiert von ‘package:base’:

    I, expand.grid, unname


Lade nötiges Paket: BiocParallel

Lade nötiges Paket: ProtGenerics


Attache Paket: ‘ProtGenerics’


Das folgende Objekt ist maskiert ‘package:stats’:

    smooth



Attache Paket: ‘Spectra’


Das folgende Objekt ist m

In [3]:
x = "/Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/Ponder_Bryo_results_RUN1_R/pos_02_spring_Fistax_2_1-A-2_01_17276.mzML"
Spectra(x, backend = MsBackendMzR())

Warning message in fun(libname, pkgname):
“mzR has been built against a different Rcpp version (1.0.8.3)
than is installed on your system (1.0.9). This might lead to errors
when loading mzR. If you encounter such issues, please send a report,
including the output of sessionInfo() to the Bioc support forum at 
https://support.bioconductor.org/. For details see also
https://github.com/sneumann/mzR/wiki/mzR-Rcpp-compiler-linker-issue.”


ERROR: Error in .local(object, ...): File(s) /Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/Ponder_Bryo_results_RUN1_R/pos_02_spring_Fistax_2_1-A-2_01_17276.mzML not found


In [ ]:
# load the functions file
source(paste(getwd(), "/Workflow_R_Functions.r", sep = ""))

In [ ]:
input_table <- data.frame(ms2_rfilename(input_dir))
input_table

In [ ]:
for (i in 1:nrow(input_table)){
    
    #Preprocess and Read the mzMLfiles
    spec_pr <- spec_Processing(input_dir,
                               input_table[i, "mzml_files"], 
                               input_table[i, "ResultFileNames"])
    
    #perform dereplication with all dbs
    df_derep <- spec_dereplication_file(mzml_file = input_table[i, "mzml_files"],
                                   pre_tbl = paste(input_dir, str_remove(paste(input_table[i, "ResultFileNames"], "/premz_list.txt", sep = ""), "."), sep =""), 
                                   proc_mzml = paste(input_dir, str_remove(paste(input_table[i, "ResultFileNames"], "/processedSpectra.mzML", sep = ""), "."), sep =""),
                                   db = "all", 
                                   result_dir = input_table[i, "ResultFileNames"],
                                   file_id = input_table[i, "File_id"], 
                                   input_dir, 
                                   no_of_candidates = 30,
                                   ppmx = 15)
    
    
    # Extract MS2 peak lists
    spec_pr2 <- ms2_peaks(pre_tbl = paste(input_dir, str_remove(paste(input_table[i, "ResultFileNames"], "/premz_list.txt", sep = ""), "."), sep =""), 
                          proc_mzml = paste(input_dir, str_remove(paste(input_table[i, "ResultFileNames"], "/processedSpectra.mzML", sep = ""), "."), sep =""),
                          input_dir,
                          result_dir = input_table[i, "ResultFileNames"],
                         file_id = input_table[i, "File_id"])
    
    # camera results for isotopes
    cam_res <- cam_func(input_dir,
                        f = input_table[i, "mzml_files"], 
                        ms2features = paste(input_dir, str_remove(paste(input_table[i, "ResultFileNames"], "/insilico/MS2DATA.csv", sep = ""), "."), sep = ""))
    
    # Extract MS1 peaks or isotopic peaks
    ms1p <- ms1_peaks(x = paste(input_dir, str_remove(paste(input_table[i, "ResultFileNames"],'/insilico/MS2DATA.csv', sep = ""), "."), sep =""), 
                      y = paste(input_dir, str_remove(paste(input_table[i, "ResultFileNames"],'/CAMERAResults.csv', sep = ""), "."), sep =""), 
                      input_table[i, "ResultFileNames"], 
                      input_dir, 
                      QCfile = TRUE)
    
    #prepare sirius parameter files
    sirius_param_files <- sirius_param(x = paste(input_dir, str_remove(paste(input_table[i, "ResultFileNames"],'/insilico/MS1DATA.csv', sep = ""), "."), sep =""), 
                                       result_dir = input_table[i, 'ResultFileNames'], 
                                       input_dir,
                                       SL = FALSE)
    
    
    # Run sirius
    run_sirius(files = paste(input_dir, str_remove(paste(input_table[i, "ResultFileNames"],'/insilico/MS1DATA_SiriusP.tsv', sep = ""), "."), sep =""), 
               ppm_max = 5, 
               ppm_max_ms2 = 15, 
               QC = TRUE, 
               SL = FALSE, 
               SL_path = NA,
               candidates = 30,
              profile = "qtof")
}

In [ ]:
end.time <- Sys.time()

In [ ]:
time.taken <- end.time - start.time
print(time.taken)